### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

#### nhc covid data

In [7]:
url = "https://weekly.chinacdc.cn/news/TrackingtheEpidemic.htm"

In [8]:
html = requests.get(url)

In [9]:
page = html.text

In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, "html.parser")

In [11]:
dates = []
for a in soup.select('a[href^="#NHC22"]'):
    dates.append(a["href"])

In [12]:
import re

In [13]:
ptags_a = soup.find_all("p", text=re.compile("Asymptomatic*"))

In [14]:
ptags_c = soup.find_all("p", text=re.compile("Confirmed*"))

In [15]:
asymptomatic_txt = []
for p in ptags_a:
    asymptomatic_txt.append(p.text.strip())

In [16]:
confirmed_txt = []
for p in ptags_c:
    confirmed_txt.append(p.text.strip())

In [17]:
df = pd.DataFrame(
    {
        "date_txt": dates,
        "year": "2022",
        "asymptomatic_text": asymptomatic_txt,
        "confirmed_text": confirmed_txt,
    }
)

In [18]:
df["monthday"] = df["date_txt"].str.replace("#NHC22", "")

In [19]:
df["date"] = df["monthday"].astype(str) + ", " + df["year"]

In [20]:
df["date"] = df["date"].str.replace("July", "Jul")
df["date"] = df["date"].str.replace("June", "Jun")

In [21]:
df["date"] = pd.to_datetime(df["date"], format="%b%d, %Y")

In [22]:
df = df[["date", "asymptomatic_text", "confirmed_text"]]

In [23]:
df[["new_a", "current_a"]] = df["asymptomatic_text"].str.split(" new,", 1, expand=True)

In [24]:
df[["new_c", "current_c"]] = df["confirmed_text"].str.split(" new,", 1, expand=True)

In [25]:
df_slim = df[["date", "new_a", "new_c"]]

In [26]:
df_slim["asymptomatic_cases"] = (
    df_slim["new_a"]
    .str.replace("Asymptomatic: ", "")
    .str.replace(",", "")
    .astype(float)
)

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_15399/2632907431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slim["asymptomatic_cases"] = (


In [27]:
df_slim["confirmed_cases"] = (
    df_slim["new_c"].str.replace("Confirmed: ", "").str.replace(",", "").astype(float)
)

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_15399/2455201535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slim["confirmed_cases"] = (


In [28]:
df_slim = df_slim.drop(columns=["new_a", "new_c"]).sort_values("date", ascending=True)

In [29]:
df_slim["total_new_cases"] = df_slim["asymptomatic_cases"] + df_slim["confirmed_cases"]

In [30]:
df_slim["rollmean"] = df_slim["total_new_cases"].rolling(7).mean().round(0)

In [31]:
alt.Chart(df_slim).mark_line().encode(x="date", y="total_new_cases")

alt.Chart(...)

In [32]:
alt.Chart(df_slim).mark_line().encode(x="date", y="rollmean")

alt.Chart(...)

In [33]:
# dw.create_chart("China covid cases", chart_type="d3-lines", data=df_slim)
# chartid='D43rq'

In [34]:
dw.add_data(chart_id="D43rq", data=df_slim)

<Response [204]>

In [35]:
df_slim

,date,asymptomatic_cases,confirmed_cases,total_new_cases,rollmean
254,2022-01-01,38.0,231.0,269.0,NaN
253,2022-01-02,52.0,191.0,243.0,NaN
252,2022-01-03,35.0,161.0,196.0,NaN
251,2022-01-04,54.0,175.0,229.0,NaN
250,2022-01-05,71.0,91.0,162.0,NaN
249,2022-01-06,45.0,189.0,234.0,NaN
248,2022-01-07,45.0,174.0,219.0,222.0
247,2022-01-08,52.0,159.0,211.0,213.0
246,2022-01-09,46.0,165.0,211.0,209.0
245,2022-01-10,42.0,157.0,199.0,209.0
